In [1]:
# import
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)import re
import fnmatch    # for later text matching
import matplotlib.pyplot as plt    # for plotting
import bokeh
from IPython.display import HTML
from IPython.core.display import display, HTML
import json

from IPython.core.display import HTML


In [2]:
import random
import inspect, os
from string import Template


def this_dir():
    this_file = inspect.getfile(inspect.currentframe())
    return os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))


def set_styles(css_file_names):
    if type(css_file_names) == str:
        style = open(this_dir() + '/css/' + css_file_names + '.css','r').read()
    else:
        style = ''
        for css_file_name in css_file_names:
            style += open(this_dir() + '/css/' + css_file_name + '.css','r').read()
    return "<style>" + style + "</style>"


def draw_graph(type, data_dict):

    JS_text = Template('''
    
                <div id='maindiv${divnum}'></div>
                
                <script>
                    $main_text
                </script>
                ''')

    divnum = int(random.uniform(0,9999999999))
    data_dict['divnum'] = divnum
    main_text_template = Template( open(this_dir() + '/js/' + type + '.js','r').read() )
    main_text = main_text_template.safe_substitute(data_dict)

    return JS_text.safe_substitute({'divnum': divnum, 'main_text': main_text})

In [23]:
colnames = ['Fit_Date', 'Leg_Dep_Date', 'Carrier_Code', 'Fit_Number', 'Leg_Orig_Apt', 'Leg_Dest_Apt', 'Leg_Orig_Port',
         'Leg_Dest_Port', 'Sch_Dep_Datetime', 'Sch_Arv_Datetime', 'Act_Dep_Datetime', 'Act_Arv_Datetime', 'Ac_Type_Code'
         'Ac_Regn_Num', 'Seat_Capacity', 'Ttl_Pax_Flown']

dataset = pd.read_csv("CSV.csv")
cols = ['Leg Orig Apt', 'Leg Dest Apt', 'Leg Dep Date' ]
dataset = dataset[cols]
dataset.cols  = ['origin', 'destination', 'date']

dataset = dataset.rename(index=str, columns={"Leg Orig Apt": "origin", "Leg Dest Apt": "destination", 'Leg Dep Date':'date'})
n_nodes = list(set(list(dataset.origin) + list(dataset.destination)))

dataset['value'] = 1

dataset = dataset[['origin','destination','value']].groupby(['origin','destination']).sum().reset_index()

In [24]:
dataset.head()

,origin,destination,value
0,ADL,HKG,148
1,ADL,PER,1
2,AKL,HKG,319
3,AMS,HKG,236
4,ANC,HKG,1


In [26]:
nodes=[]
for elem in n_nodes:
        nodes.append({'name': elem, 'group':n_nodes.index(elem)})
        
graph = {"nodes": nodes, "links": []}

for index , row in dataset.iterrows():
    graph["links"].append( {
        "source": n_nodes.index(row['origin']), 
        "target": n_nodes.index(row['destination']), 
        "value": row['value']
    } )
    
    
   

In [30]:
%%javascript
require.config({
  paths: {
    d3: "http://d3js.org/d3.v3.min"
  }
});

<IPython.core.display.Javascript object>

In [31]:
HTML( set_styles(['force_directed_graph']) \
        + '<script src="lib/d3/d3.min.js"></script>' \
        +  draw_graph('force_directed_graph', {'data': graph}) 
)

In [8]:

# construct graph as a set of nodes and links between them
n_nodes = 30
p_edge = 0.05
graph = {"nodes": [], "links": []}
for i in range(n_nodes):
    graph["nodes"].append( {"name": "i" + str(i), "group": int(random.uniform(1,11))} )
for i in range(n_nodes):
    for j in range(n_nodes):
        if random.uniform(0,1) < p_edge and i != j:
            graph["links"].append( {"source": i, "target": j,
                             "value": random.uniform(0.5,3)} )